In [2]:
import pandas as pd
import shutil
import os

In [3]:
%%bash
git clone https://github.com/Jaehun-Kim22/SCIC_Project.git

Cloning into 'SCIC_Project'...


In [18]:
## loads excel
xls = pd.ExcelFile('SCIC_Project/assets/공모전 제공 데이터(2차)_210804.xlsx')
training_data = pd.read_excel(xls, '학습데이터')
priority = pd.read_excel(xls, '우선순위')
xls_prev = pd.ExcelFile('SCIC_Project/assets/data.xlsx')
training_data_2 = pd.read_excel(xls_prev, '학습데이터')
training_data = training_data.append(training_data_2)
print(training_data)
# data_without_ground_truth = training_data.values[:, 2]
# !pip install konlpy

       NO    NO2  ... 발화3의 우선순위 최종분류(우선순위 가장 높은것 선택)
0     126  25068  ...       NaN       불만>고객서비스>상담시스템
1     127  25078  ...       NaN       불만>고객서비스>상담시스템
2     128  25079  ...       NaN       불만>고객서비스>상담시스템
3     129  25081  ...       NaN       불만>고객서비스>상담시스템
4     130  25083  ...       NaN       불만>고객서비스>상담시스템
..    ...    ...  ...       ...                  ...
995  9958    103  ...       NaN         칭찬>삼성카드>카드상품
996  9961      2  ...       NaN           칭찬>삼성카드>혜택
997  9962      4  ...       NaN           칭찬>삼성카드>혜택
998  9963      5  ...       NaN           칭찬>삼성카드>혜택
999  9964      7  ...       NaN           칭찬>삼성카드>혜택

[10000 rows x 10 columns]


In [73]:
priority_dict = {}

for i in range(len(priority['의도명(유형)'])):
  priority_dict[priority['의도명(유형)'][i]] = priority['우선순위'][i]
print(priority_dict)

label_level3 = {}
label_count = {}
level3_cnt = 1
label_count['중립'] = 0
label_level3['중립'] = 0
for i in priority['의도명(유형)']:
  level = i.split('>')
  if len(level) == 3 and level[2] not in label_level3:
    label_level3[level[2]] = level3_cnt
    label_count[level[2]] = 0
    level3_cnt += 1
  elif len(level) == 2 and level[1] not in label_level3:
    label_level3[level[1]] = level3_cnt
    label_count[level[1]] = 0
    level3_cnt += 1
print(label_level3)
print(label_level3.keys())
print(label_count)
label_count_for_split = label_count.copy()
label_sentiment = {'칭찬': 0, '불만': 1}


{'칭찬>고객서비스>상담원': 1, '칭찬>고객서비스>상담시스템': 2, '칭찬>고객서비스': 3, '불만>고객서비스>상담원': 4, '불만>고객서비스>상담시스템': 5, '불만>고객서비스': 6, '칭찬>삼성카드>혜택': 7, '칭찬>삼성카드>할부금융상품': 8, '칭찬>삼성카드>커뮤니티서비스': 9, '칭찬>삼성카드>카드이용/결제': 10, '칭찬>삼성카드>카드상품': 11, '칭찬>삼성카드>청구입금': 12, '칭찬>삼성카드>심사/한도': 13, '칭찬>삼성카드>생활편의서비스': 14, '칭찬>삼성카드>상담/채널': 15, '칭찬>삼성카드>리스렌탈상품': 16, '칭찬>삼성카드>라이프서비스': 17, '칭찬>삼성카드>금융상품': 18, '칭찬>삼성카드>고객정보관리': 19, '칭찬>삼성카드>가맹점매출/승인': 20, '칭찬>삼성카드>가맹점대금': 21, '칭찬>삼성카드>가맹점계약': 22, '칭찬>삼성카드': 23, '칭찬>기타': 24, '불만>삼성카드>혜택': 25, '불만>삼성카드>할부금융상품': 26, '불만>삼성카드>커뮤니티서비스': 27, '불만>삼성카드>카드이용/결제': 28, '불만>삼성카드>카드상품': 29, '불만>삼성카드>청구입금': 30, '불만>삼성카드>심사/한도': 31, '불만>삼성카드>생활편의서비스': 32, '불만>삼성카드>상담/채널': 33, '불만>삼성카드>리스렌탈상품': 34, '불만>삼성카드>라이프서비스': 35, '불만>삼성카드>금융상품': 36, '불만>삼성카드>고객정보관리': 37, '불만>삼성카드>가맹점매출/승인': 38, '불만>삼성카드>가맹점대금': 39, '불만>삼성카드>가맹점계약': 40, '불만>삼성카드': 41, '불만>기타': 42, '중립': 43, '폐기': 44}
{'중립': 0, '상담원': 1, '상담시스템': 2, '고객서비스': 3, '혜택': 4, '할부금융상품': 5, '커뮤니티서비스': 6, '카드이용/결제': 7, '카드상품': 8, '청구입금': 9, '심사/한도': 10, '

In [36]:
import requests
import json
import sys
import re
import xml.etree.ElementTree as ET
import copy

base_url = 'https://m.search.naver.com/p/csearch/ocontent/spellchecker.nhn'

_agent = requests.Session()
PY3 = sys.version_info[0] == 3


def _remove_tags(text):
    text = u'<content>{}</content>'.format(text).replace('<br>', '')
    if not PY3:
        text = text.encode('utf-8')

    result = ''.join(ET.fromstring(text).itertext())

    return result


def check(text):

    if isinstance(text, list):
        result = []
        for item in text:
            checked = check(item)
            result.append(checked)
        return result

    payload = {
        '_callback': 'window.__jindo2_callback._spellingCheck_0',
        'q': text
    }

    headers = {
        'user-agent': 'https://github.com/michael960725',
        'referer': 'https://naver.com/',
    }

    r = _agent.get(base_url, params=payload, headers=headers)
    r = r.text[42:-2]

    data = json.loads(r)
    html = data['message']['result']['html']
    return _remove_tags(html)


parsed_text_list = []
training_data_deepcopy = copy.deepcopy(training_data)
naver = 0
for i in training_data_deepcopy['발화']:
  naver += 1
  if naver % 100 = 0:
    print(naver)
  new_text = re.sub('[^가-힣a-zA-Z0-9]', " ", i)
  parsed_text_list.append(check(new_text))

In [ ]:
# training_data_copy_kkma = copy.deepcopy(training_data)
label_sentiment = {'칭찬': 0, '중립': 'NAN', '불만': 1}
cnt = 0
column_label_level3 = []
column_label_sentiment = []
column_ground_truth = []
delete_neutral = False
parsed_text_wo_neutral = copy.deepcopy(parsed_text_list)
if len(parsed_text_list) == len(parsed_text_wo_neutral):
  delete_neutral = True
for ground_truth in training_data['최종분류(우선순위 가장 높은것 선택)']:
  level = ground_truth.split('>')
  if len(level) == 3:
    # column_label_level3.append(label_level3[level[2]])
    # column_label_sentiment.append(label_sentiment[level[0]])
    column_label_level3.append(level[2])
    column_label_sentiment.append(level[0])
    label_count[level[2]] += 1
  elif len(level) == 2:
    # column_label_level3.append(label_level3[level[1]])
    # column_label_sentiment.append(label_sentiment[level[0]])
    column_label_level3.append(level[1])
    column_label_sentiment.append(level[0])
    label_count[level[1]] += 1
  else:
    if delete_neutral and level == ['중립']:
      del parsed_text_wo_neutral[cnt]
    # del parsed_text_wo_neutral[cnt]
    label_count[level[0]] += 1
    # column_label_level3.append(label_level3[level[0]])
    column_label_level3.append(level[0])
  column_ground_truth.append(priority_dict[ground_truth])
  cnt += 1
print(len(column_label_level3))
print(len(parsed_text_wo_neutral))
data = {'발화': parsed_text_list, 'label': column_label_level3}
data_sentiment = {'발화': parsed_text_wo_neutral, 'label': column_label_sentiment}
data_ground_truth = {'발화': parsed_text_list, 'label': column_ground_truth}
label_for_level3 = pd.DataFrame(data)
label_for_sentiment = pd.DataFrame(data_sentiment)
label_for_ground_truth = pd.DataFrame(data_ground_truth)
# df = pd.DataFrame(np.random.randn(100, 2))
print(label_for_level3)
print(label_for_sentiment)
# training_data_copy_kkma['label'] = column_label_level3
# print(training_data_copy_kkma['최종분류(우선순위 가장 높은것 선택)'], training_data_copy_kkma['label'])


In [76]:
print(label_level3)
print(label_count)
new_dict = {'중립': 0, '상담원': 1, '상담시스템': 2, '혜택': 3, '할부금융상품': 4, 
            '카드상품': 5, '청구입금': 6, '심사/한도': 7, '생활편의서비스': 8, 
            '상담/채널': 9, '리스렌탈상품': 10, '라이프서비스': 11, '금융상품': 12, 
            '고객정보관리': 13, '가맹점매출/승인': 14, '삼성카드': 15, '기타': 16}
# for i in (k, v) in label_count.items():
#   if v == 0:


{'중립': 0, '상담원': 1, '상담시스템': 2, '고객서비스': 3, '혜택': 4, '할부금융상품': 5, '커뮤니티서비스': 6, '카드이용/결제': 7, '카드상품': 8, '청구입금': 9, '심사/한도': 10, '생활편의서비스': 11, '상담/채널': 12, '리스렌탈상품': 13, '라이프서비스': 14, '금융상품': 15, '고객정보관리': 16, '가맹점매출/승인': 17, '가맹점대금': 18, '가맹점계약': 19, '삼성카드': 20, '기타': 21}
{'중립': 70, '상담원': 4545, '상담시스템': 1600, '고객서비스': 0, '혜택': 260, '할부금융상품': 6, '커뮤니티서비스': 0, '카드이용/결제': 0, '카드상품': 23, '청구입금': 360, '심사/한도': 21, '생활편의서비스': 55, '상담/채널': 1410, '리스렌탈상품': 1, '라이프서비스': 20, '금융상품': 4, '고객정보관리': 135, '가맹점매출/승인': 0, '가맹점대금': 0, '가맹점계약': 0, '삼성카드': 190, '기타': 1300}


In [82]:
print(label_count_for_split)
print(label_count)
reverse_dict = dict((v, k) for (k, v) in label_level3.items())
print(reverse_dict)
insert_df = {'Review': [], 'Label': []}
train_cls = pd.DataFrame(insert_df)
print(train_cls)
validation_cls = pd.DataFrame(insert_df)
test_cls = pd.DataFrame(insert_df)
count123 = 0
for review, label in label_for_level3.values[:, 0:2]:
  count_for_split = label_count_for_split[label]
  if count_for_split < label_count[label] * 0.8:
    train_cls.append({'Review': review, 'Label': label}, ignore_index=True)
  elif count_for_split < label_count[label] * 0.9:
    validation_cls.append({'Review': review, 'Label': label}, ignore_index=True)
  else:
    print(count123)
    test_cls.append({'Review': review, 'Label': label}, ignore_index=True)
  label_count_for_split[label] += 1
  count123 += 1


{'중립': 0, '상담원': 484, '상담시스템': 1125, '고객서비스': 0, '혜택': 0, '할부금융상품': 0, '커뮤니티서비스': 0, '카드이용/결제': 0, '카드상품': 0, '청구입금': 0, '심사/한도': 0, '생활편의서비스': 0, '상담/채널': 0, '리스렌탈상품': 0, '라이프서비스': 0, '금융상품': 0, '고객정보관리': 0, '가맹점매출/승인': 0, '가맹점대금': 0, '가맹점계약': 0, '삼성카드': 0, '기타': 0}
{'중립': 70, '상담원': 4545, '상담시스템': 1600, '고객서비스': 0, '혜택': 260, '할부금융상품': 6, '커뮤니티서비스': 0, '카드이용/결제': 0, '카드상품': 23, '청구입금': 360, '심사/한도': 21, '생활편의서비스': 55, '상담/채널': 1410, '리스렌탈상품': 1, '라이프서비스': 20, '금융상품': 4, '고객정보관리': 135, '가맹점매출/승인': 0, '가맹점대금': 0, '가맹점계약': 0, '삼성카드': 190, '기타': 1300}
{0: '중립', 1: '상담원', 2: '상담시스템', 3: '고객서비스', 4: '혜택', 5: '할부금융상품', 6: '커뮤니티서비스', 7: '카드이용/결제', 8: '카드상품', 9: '청구입금', 10: '심사/한도', 11: '생활편의서비스', 12: '상담/채널', 13: '리스렌탈상품', 14: '라이프서비스', 15: '금융상품', 16: '고객정보관리', 17: '가맹점매출/승인', 18: '가맹점대금', 19: '가맹점계약', 20: '삼성카드', 21: '기타'}
Empty DataFrame
Columns: [Review, Label]
Index: []
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
34

KeyboardInterrupt: ignored

In [77]:
print(test_cls)

Empty DataFrame
Columns: [Review, Label]
Index: []


In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(label_for_level3, test_size=0.2)
train_s, test_s = train_test_split(label_for_sentiment, test_size = 0.2)
train_g, test_g = train_test_split(label_for_ground_truth, test_size = 0.2)

In [11]:
import numpy as np
train.to_csv(r'train.txt', header=None, index=None, sep='\t', mode='a')
test.to_csv(r'test.txt', header=None, index=None, sep='\t', mode='a')
train_s.to_csv(r'train_s.txt', header=None, index=None, sep='\t', mode='a')
test_s.to_csv(r'test_s.txt', header=None, index=None, sep='\t', mode='a')
train_g.to_csv(r'train_g.txt', header=None, index=None, sep='\t', mode='a')
test_g.to_csv(r'test_g.txt', header=None, index=None, sep='\t', mode='a')
# np.savetxt(r'c:Wtest.txt', test.values, fmt='%s')
# 이 파일들은 올려만 놓고 참조하는 방식

In [ ]:
## Kkma sentence
from konlpy.tag import Kkma
import copy
import time
kkma = Kkma()


def perform_Kkma(text):
  temp_kkma = kkma.sentences(text)
  if [text] != temp_kkma:
    new_string_to_add = ''
    new_string_to_add += ' '.join(temp_kkma)
    return new_string_to_add
  else:
    return text


s = time.time()

training_data_copy_kkma = copy.deepcopy(training_data)

cnt = 0
column_added_kkma = []
for text in training_data['발화']:
  column_added_kkma.append(perform_Kkma(text))
  cnt += 1
training_data_copy_kkma['Kkma'] = column_added_kkma

print(training_data.values[:, 2][:20])
print(training_data_copy_kkma.values[:, 10][:20])
e = time.time()

print(e - s)

['상담직원과 빠른연결했슴좋겠어요' '안기다리고. 빨리상담되었음좋겠다' '너무 오래기다리고 상담이 늦어져요'
 '상담직원과  통화가  매우  힘들다' '상담받으려면오래기다리는게불편함' '상담하려면 많이기다려야하는게 불편합니다.'
 '상담원이통화중이라많이기다려야해요' '상담직원과의연결이 빠르게연결되면촣겠습니다' 'ars상담직원넘많이기다려야하는불편해요'
 '상담원 통화가너믜힘듬' '상담원과의 연결이빨리되었으면감사힘니다' '상담원이 영결이많이힘이듬'
 '5번질문에 대한 답변은 상담직원 연결되기까지 기다리는것말곤없는듯'
 '상담을 하려면 좀 많이기다려야 상담사분이 나오셔서 그게좀 불편하네요' 'Talk 상담이 다시 생겼으면..'
 '상담하려하는  기다리는시간    너무길다' '상담직원과 빠른속도로  연결이되길 원합니다' '상담직원과 빠른연결부탁드립니다'
 '상담직원과통화하기가힘들다' '상담한번받을려면 보통5분이상대기해야실질상담은2분미만인데.']
['상담 직원과 빠른연결했슴좋겠어요' '안 기다리고. 빨리 상담되었음 좋겠다' '너무 오래 기다리고 상담이 늦어져요'
 '상담 직원과 통화가 매우 힘들다' '상담 받으려면 오래 기다리는 게 불편함' '상담하려면 많이 기다려야 하는 게 불편합니다.'
 '상담원이 통화 중이라 많이 기다려야 해요' '상담 직원 과의 연결이 빠르게 연결되면 촣겠습니다'
 'ars 상담 직원 넘 많이 기다려야 하는 불편해요' '상담원 통화가 너 믜 힘 듬'
 '상담원과의 연결이 빨리 되었으면 감사 힘니다' '상담원이 영결이 많이 힘이 듬'
 '5번 질문에 대한 답변은 상담 직원 연결되기까지 기다리는 것 말곤 없는 듯'
 '상담을 하려면 좀 많이 기다려야 상담 사분이 나오셔서 그게 좀 불편하네요' 'Talk 상담이 다시 생겼으면..'
 '상담하려 하는 기다리는 시간 너무 길다' '상담 직원과 빠른 속도로 연결이 되길 원합니다' '상담 직원과 빠른 연결 부탁드립니다'
 '상담 직원과 통화하기가 힘들다' '상담한 번받 을려 면 보통 5분 이상 대기해야 실질

In [ ]:
!pip install transformers
from transformers import BertModel, ElectraTokenizer, BertTokenizer
model = BertModel.from_pretrained('skt/kobert-base-v1')
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
print(tokenizer.tokenize("삼성카드가 제일 전문적이고 정확하다."))
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer.tokenize('삼성카드가 제일 전문적이고 정확하다.')

['삼성카드', '##가', '제일', '전문', '##적', '##이', '##고', '정확', '##하', '##다', '.']


['삼',
 '##성',
 '##카',
 '##드',
 '##가',
 '제',
 '##일',
 '전',
 '##문',
 '##적',
 '##이고',
 '정',
 '##확',
 '##하다',
 '.']

In [ ]:
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer.decode([2,11029, 6851, 17788, 3])

'[CLS] 친절 감사합니다 [SEP]'